# Дисклеймер
Это вырезанный из общей системы код. По факту класс Robot выводит вас на биржу, но в данной версии заявки не уходят никуда, они остаются внутри робота и исполняются чисто формально. Во всех местах, где робот должен получать информацию от другого объекта, стоят заглушки в виде параметров по умолчанию. В данной версии вы можете протестировать, что вы получаете те данные, которые хотите, и отправляете заявки в правильном формате.

In [33]:
# эту клетку надо запустить, чтобы можно было попробовать своего робота
from abc import ABC, abstractmethod
import pandas as pd
import datetime as dt


class Order:
    def __init__(self,
                 operation: 'buy or sell',
                 order_type: 'limit or datasource',
                 lots: int,
                 price: float = None,
                 username: str = None,
                 datetime: 'datetime.datetime' = None,
                 is_by_robot: bool = False,
                 order_no: int = None,
                 to_delete: bool = False):
        self.operation = operation
        self.order_type = order_type
        self.lots = lots
        self.price = price
        self.username = username
        self.datetime = datetime
        self.is_by_robot = is_by_robot
        self.order_no = order_no
        self.to_delete = to_delete

    def __str__(self):
        return f'Order({self.operation}, {self.order_type}, ' \
               f'{self.lots}, {self.price})'

    def __repr__(self):
        return f'Order({self.operation}, {self.order_type}, {self.lots}, ' \
               f'price={self.price}, username={self.username}, ' \
               f'datetime={self.datetime.strftime("%H:%M:%S.%f")}, ' \
               f'is_by_robot={self.is_by_robot}, order_no={self.order_no}, ' \
               f'to_delete={self.to_delete}'


class Robot(ABC):
    def __init__(self,
                 username: str):
        self.username = username
        self.df = None

        self.orders = []
        self.data_from_order_book = None
        self.data_from_accountant = None
        self.new_actions = []

    def set_hist_prices(self,
                        df: 'pd.DataFrame'):
        self.df = df

    def update(self,
               data_from_order_book: dict,
               orders: list,
               data_from_accountant: dict):
        self.data_from_order_book = data_from_order_book
        self.orders = orders
        self.data_from_accountant = data_from_accountant

    def get_bid(self,
                volume_too: bool = False) -> float or ():
        if volume_too:
            return self.data_from_order_book['max_bid_price'], \
                   self.data_from_order_book['max_bid_volume']
        return self.data_from_order_book['max_bid_price']

    def get_ask(self,
                volume_too: bool = False) -> float or ():
        if volume_too:
            return self.data_from_order_book['min_ask_price'], \
                   self.data_from_order_book['min_ask_volume']
        return self.data_from_order_book['min_ask_price']

    def get_last_quote(self):
        return self.data_from_order_book['last_quote']

    def get_last_quote_time(self):
        return self.data_from_order_book['datetime']

    def get_current_balance(self):
        return self.data_from_accountant['balance']

    def get_current_net_position(self) -> (float, int):
        return self.data_from_accountant['cnp_price'], \
               self.data_from_accountant['cnp_lots']

    @staticmethod
    def format_price(price: float):
        return float(format(price, '.2f'))

    def get_active_orders(self):
        return self.orders

    def get_buy_orders(self):
        buy_orders = []
        for order in self.orders:
            if order.operation == 'buy':
                buy_orders.append(order)
        return buy_orders

    def get_sell_orders(self):
        sell_orders = []
        for order in self.orders:
            if order.operation == 'sell':
                sell_orders.append(order)
        return sell_orders

    def order_send(self,
                   operation: 'buy or sell',
                   order_type: 'limit or datasource',
                   lots: int,
                   price: float = None):
        if order_type == 'datasource':
            self.data_from_accountant['cnp_price'] = self.get_ask() if operation == 'buy' else self.get_bid()
            self.data_from_accountant['cnp_lots'] += lots
        elif order_type == 'limit':
            if operation == 'buy' and price >= self.get_ask() or operation == 'sell' and price <= self.get_bid():
                self.data_from_accountant['cnp_price'] = self.get_ask() if operation == 'buy' else self.get_bid()
                self.data_from_accountant['cnp_lots'] += lots
            else:
                self.orders.append(Order(operation,
                                   order_type,
                                   lots,
                                   self.format_price(price),
                                   self.username,
                                   self.data_from_order_book['datetime'],
                                   True,
                                   False))

    def order_delete(self,
                     order):
        for i in range(len(self.orders)):
            if self.orders[i] is order:
                self.orders.pop(i)
                return

    def order_change(self,
                     order,
                     price: float = None,
                     lots: float = None):
        order.price = price or order.price
        order.lots = lots or order.lots

    @abstractmethod
    def train(self):
        pass

    @abstractmethod
    def on_tick(self):
        pass


def test(robot: Robot):
    robot.set_hist_prices(pd.DataFrame(
        {'time': ['10:00:00', '10:01:00', '10:02:00', '10:03:00', '10:04:00',
                  '10:05:00', '10:06:00', '10:07:00', '10:08:00', '10:09:00',
                  '10:10:00', '10:11:00', '10:12:00', '10:13:00', '10:14:00',
                  '10:15:00', '10:16:00', '10:17:00', '10:18:00', '10:19:00'],
         'open': [103.45, 103.19, 103.29, 103.48, 103.43, 103.31, 103.25,
                  103.39, 103.31, 103.2, 103.06, 103.16, 103.12, 103.08,
                  102.88, 103.06, 103.04, 102.97, 103.26, 103.23],
         'high': [103.53, 103.29, 103.49, 103.54, 103.45, 103.38, 103.45, 103.4,
                 103.31, 103.2, 103.2, 103.19, 103.15, 103.15, 103.06, 103.14,
                 103.05, 103.27, 103.31, 103.34],
         'low': [103.08, 103.15, 103.25, 103.4, 103.3, 103.19, 103.25, 103.3,
                 103.17, 102.98, 103.05, 103.08, 103.06, 102.86, 102.85,
                 103.01, 102.91, 102.97, 103.2, 103.21],
         'close': [103.2, 103.29, 103.48, 103.44, 103.33, 103.27, 103.35,
                   103.32, 103.2, 103.08, 103.15, 103.12, 103.07, 102.86,
                   103.06, 103.04, 102.97, 103.25, 103.23, 103.29],
         'volume': [637240, 419940, 560950, 165420, 417730, 526180, 627840,
                    249160, 286250, 451580, 128290, 70270, 97870, 746310,
                    377610, 101500, 373370, 267130, 118550, 320600],
         'day': [1] * 20}
    ))

    data = ({'max_bid_price': 103.2,
             'max_bid_volume': 100000,
             'min_ask_price': 103.22,
             'min_ask_volume': 100000,
             'datetime': dt.datetime(hour=9, minute=59, second=59, year=2020, month=12, day=10),
             'last_quote': 103.21,},
             [],
             {'balance': 100000,
              'cnp_price': 0,
              'cnp_lots': 0})
    robot.update(*data)
    robot.train()
    for i in range(60):
        robot.data_from_order_book['datetime'] += dt.timedelta(seconds=1)
        robot.on_tick()


# Все методы и поля у робота, которыми вы можете пользоваться

## Методы

#### Получение биржевых данных
* **get_bid(volume_too: bool = False) -> float or (float, int)** - возвращает лучшую цену спроса на текущий момент (1 число типа float), если указать в скобочках True, вернет еще и объем, который можно приобрести по этой цене (в таком случае возвращается кортеж)
* **get_ask(volume_too: bool = False) -> float or (float, int)** - возвращает лучшую цену предложения на текущий момент (1 число типа float), если указать в скобочках True, вернет еще и объем, который можно продать по этой цене (в таком случае возвращается кортеж)
* **get_last_quote() -> float** - возвращает цену последней сделки на текущий момент (1 число типа float)
* **get_last_quote_time() -> datetime.datetime** - возвращает время последней сделки (оно с каждым тиком будет увеличиваться на 1 секунду). У объекта, который возвращается функцией, вам могут понадобиться поля (к ним просто обращаемся через точку и без скобочек в конце): *hour: int* - час, *minute: int* - минута, *second: int* - секунда

#### Получение данных по счету
* **get_current_balance() -> float** - возвращает текущий баланс (каждую секунду он, естественно, пересчитывается, если у вас открыта позиция)
* **get_current_net_position() -> (float, int)** - возвращает цену текущей чистой позиции (расчет идет по методу AVCO) и объем текущей чистой позиции (если вы находитесь в шорте, будет отрицательное число) - кортеж из 2 элементов.

#### Работа с ордерами
* **order_send(operation: 'buy or sell', order_type: 'limit or datasource', lots: int, price: float = None)** - отправляет ордер на биржу. 1 параметром необходимо указать операцию: 'buy' или 'sell', 2 параметром - тип ордера: 'limit' или 'datasource', 3 параметром - объем (целое число), 4 параметром (опционально) - если отправляете лимитный ордер, указываете цену, если рыночный, не указываете ничего
* **order_delete(order: Order)** - снимает активный ордер из стакана. Необходимо передать сам ордер (который можно получить из списка активных ордеров (см. ниже)). Вы можете и не успеть снять ордер (вы отправили заявку на его снятие, но пока доходила эта заявка до биржи, ордер уже могли исполнить, тогда просто ничего не произойдет)
* **order_change(order: Order, price: int=None, lots: float: None)** - меняет параметры активного ордера. Необходимо передать сам ордер (который можно получить из списка активных ордеров (см. ниже)), помимо этого можно указать именованные параметры: *price* - новая цена ордера (если не указана, цена не изменится), *lots* - новый объем ордера (если не указан, объем не изменится). Если не указан ни один из переданных параметров, ничего не произойдет

#### Получение информации по ордерам
* **get_active_orders() -> [Order]** - возвращает список открытых ордеров (!если ордер исполнился, он отсюда пропадает, здесь хранятся только ордера, которые в текущий момент стоят в стакане). По списку можно пробежать обычным циклом for, при этом вам могут понадобиться поля у ордера: *operation: str* - 'buy' или 'sell', *price: float* - цена ордера, *lots: int* - объем ордера
* **get_buy_orders() -> [Order]** - возвращает список открытых ордеров на покупку (использование абсолютно такое же, как и в методе выше, но возвращает список только ордеров на покупку, т.е. за вас их фильтрует)
* **get_buy_orders() -> [Order]** - возвращает список открытых ордеров на продажу (использование абсолютно такое же, как и в методе выше, но возвращает список только ордеров на продажу, т.е. за вас их фильтрует)

## Поля
* df: pd.DataFrame - датафрейм, в котором будут лежать исторические цены (из hist_prices.csv)

После внимательного прочтения инструкции, вы готовы написать своего робота. Для этого надо всего лишь написать 2 метода внутри уже своего класса, отнаследованного от Robot: 
* **train(self)** - здесь вы проводите бэктест и превдарительную настройку всех необходимых параметров своего робота
* **on_tick(self)** - здесь вы пишите торговые правила вашего робота (также можно и перенастраиваться в процессе)

# Пример

Важно: пример придуман за 5 минут, написан еще за 20 (не учитывая времени на комментарии), поэтому логику работы вы можете продумать гораздо лучше и оптимальнее. Главное, понимать, что train() вызывается у вас 1 раз перед начало торгов, а on_tick() - каждую секунду.

In [34]:
class MyRobot(Robot):  # отнаследовался
    def __init__(self):
        super().__init__('Тестовый робот')  # просто в ужасе скопировал эту строчку и изменил название робота
        # задаю себе параметры, которые мне надо сохранять между вызовами функции (временные данные)
        
        # ограничение: нельзя здесь определять переменные self.orders, self.df (они служебные)
        
        self.risk = 2  # риск на сделку в процентах
        self.sl = None  # стоп-лосс (т.к. пока торговать не начал, там нет ничего => None)
        self.tp = None  # тейк-профит (т.к. пока торговать не начал, там нет ничего => None)
        self.period = 5  # период скользящей средней по минутным свечкам (в данном случае грубо говоря константа)
        self.window = []  # список, в котором буду хранить цены закрытия свечей для подсчета среднего
        self.mean = None  # здесь буду хранить среднее (т.к. пока торговать не начал, там нет ничего => None)
        self.max_spread = 0.10  # это опять же константа (дальше видно, зачем)
        # флаговые переменные
        self.wait_to_open_long = False  # жду ли я открытия лонга
        self.in_long = False  # нахожусь ли сейчас в лонге
        
    def train(self):
        # здесь бэкстетируюсь 
        
        # обратился к self.df как к обычному датафрейму и взял последние 5 цен закрытия, потом преобразовал в списочек
        self.window = list(self.df['close'].iloc[-self.period:])
        # посчитал по ним среднее
        self.mean = sum(self.window) / len(self.window)
        
        # в этот момент я набэктестировался
        # но у вас может быть любой сложности код хоть с тысячей настраиваемых параметров и сотней нейросетей
        # помните о том, что данная функция запустится только 1 раз перед началом торговли в 09:59:59
    
    def on_tick(self):
        # здесь пишу торговые правила
        
        # для начала получу всю доступную информацию 
        # (вам скорее всего все сразу не понадобится, лучше получать информацию там, где непосредственно она понадобилась)
        # биржевые данные
        bid, bid_volume = self.get_bid(True)
        ask, ask_volume = self.get_ask(True)
        price = self.get_last_quote()
        time = self.get_last_quote_time()
        # данные по счету
        balance = self.get_current_balance()
        cnp_price, cnp_volume = self.get_current_net_position()
        # данные по ордерам
        all_active_orders = self.get_active_orders()
        buy_orders = self.get_buy_orders()
        sell_orders = self.get_sell_orders()
        
        # я решил, что мне много данных не надо, только цена закрытия минутной свечи
        # на 59 секунде буду обновлять окно и пересчитывать среднее
        if time.second == 59:
            self.window.pop(0)  # удаляю первый элемент окна (по времени она будет самой далекой)
            self.window.append(price)  # добавляю в конец текущую цену (по времени она будет ближайшей)
            self.mean = sum(self.window) / len(self.window)  # пересчитываю среднее
        # также я решил, что буду принимать решение об открытии позиции на 20 секунде
        if time.second == 20:
            # если последняя цена выше среднего, открою лонг
            if price > self.mean:
                # проверю спред и если он меня устроит, откроюсь по рынку
                if (ask - bid) <= self.max_spread:
                    self.sl = ask * 0.99  # откроют меня по рынку примерно по аску, поэтому стоп ставлю -1% отступ от него
                    self.tp = ask * 1.02  # аналогично определил тейк на +2% 
                    # теперь надо бы определиться с лотом, для этого допишу себе отдельную функцию (см. ниже)
                    lots = self.calc_lots(ask)
                    self.order_send('buy', 'datasource', lots)
                    self.wait_to_open_long = True  # обновлю флаговую переменную 
                # если спред большой, по рынку страшно открываться, поэтому попробую открыться лимиткой
                else:
                    open_price = bid + 0.01  # перекрою лучшую цену спроса, чтобы оказаться первым
                    # дальше все аналогично
                    self.sl = open_price * 0.99
                    self.tp = open_price * 1.02
                    lots = self.calc_lots(open_price)
                    self.order_send('buy', 'limit', lots, open_price)
                    self.wait_to_open_long = True
            # для шорта все зеркально сделаю, но переписывать желания у меня нет :)
        # я решил, что в том случае, если не удалось открыть позицию на 40 секунде, подвину заявку
        elif time.second == 40:
            if self.wait_to_open_long: # вот зачем нужна флаговая переменная: не на каждой 40 секунде я же жду открытия лонга
                # проверю как там с ордерами обстоит вопрос
                buy_orders = self.get_buy_orders()  # я пишу только про лонг, поэтому проверю только ордера на покупку
                if len(buy_orders) == 0: 
                    #  ура, меня исполнили, не забуду изменить флаги
                    self.wait_to_open_long = False
                    self.in_long = True
                else:
                    # не исполнили :(
                    for order in buy_orders:  # иду по всем ордерам на покупку (ну вдруг я открываюсь 10 ордерами с шагом 0.01)
                        new_price = order.price + 0.10  # подвину цену на 10 копеек
                        new_lots = self.calc_lots(new_price)  # пересчитаю необходимый объем (вот почему ее надо написать)
                        self.order_change(order, price=new_price, lots=new_lots)
        # если меня не исполнили даже к 55 секунде, снимаю завку
        elif time.second == 55:
            if self.wait_to_open_long:  # опять проверю флаг
                # все аналогично ниже
                buy_orders = self.get_buy_orders() 
                if len(buy_orders) == 0: 
                    #  ура, меня исполнили, не забуду изменить флаги
                    self.wait_to_open_long = False
                    self.in_long = True
                else:
                    # не исполнили :(
                    for order in buy_orders:  # иду по всем ордерам опять
                        self.order_delete(order)  # но теперь я их беспощадно удаляю такой простой строчкой
                    
            
            
    def calc_lots(self, open_price):
        # (я обязан до вызова функции обновить значение в перменной self.sl)
        balance = self.get_current_balance()  # получу баланс
        max_loss_for_1lot = abs(self.sl - open_price)  # посчитаю максимальный убыток на 1 лот 
        max_available_loss = balance * self.risk / 100  # считаю максимальный доступный по рискам убыток
        return int(max_available_loss / max_loss_for_1lot)  # возвращаю максимальный лот в формате целого числа!
        
        
        

# Протестирую, что мой робот запускается

In [35]:
robot = MyRobot()  # проинициализирую его
test(robot)  # запущу и увижу, что ошибок нет

В целом прохождение этого теста гарантирует на 90%, что вы не наделали ничего криминального и робот написан правильно. Но основное его предназначение в другом: правильно будет пошагово писать робота и тестировать каждый момент с отладочным принтом. 

Например, в функции on_tick() написать просто одну строчку print(self.get_bid()) и посмотреть, что будет на тесте. Спойлер: вы увидите бид, стоящий по умолчанию, его вы не можете изменить даже если скупите весь объем заявки (а как узнать еще и объем см. в инструкции). 

Дальше вы можете отправить заявку на продажу по этому биду и посмотреть, что произошло. Спойлер: она исполнится и вы сможете это понять, воспользовавшись методом get_current_net_position(). Или можете отправить заявку выше бида и посмотреть, что произойдет. Спойлер: если будете принтить get_sell_orders(), увидите список с вашим ордером, он всегда там, если активен.

Немного информации по тесту: 
* в self.df у вас будет лежать не полноценный датафрейм с hist_prices.csv, а только его первые 20 строк (ну это же тест все-таки)
* заявки не выводятся на биржу, на бид и аск вы повлиять не можете
* прогоняется всего 1 минута торгового времени (т.е. 60 раз вызовется ваша функция on_tick)
* если остались вопросы, можете написать в телеграм *@eura71*

# Шаблон для вашего тестового робота

In [36]:
class IWillTryToUnderstandIt(Robot):  # название класса лучше делать покороче и попонятнее
    def __init__(self):
        super().__init__('Гуру рынка')  # здесь название команды, или любой другой опознавательный знак, просто в кавычках
    
    def train(self):
        print('Тренируюсь...')
    
    def on_tick(self):
        print(f'Прошла секунда {self.get_last_quote_time().second}')

Для начала просто запустите его на тест, и вам станет понятнее, что происходит. Потом сотрите мои принты и начинайте кодить. Надеюсь, вы справитесь. Удачи!